# Scenario: ICU Patient Health Risk Forecasting
A hospital ICU continuously monitors critical patient vitals.
The goal is to predict the patient’s risk score for the next 6 hours using the previous 12 hours of multivariate time-series data, so doctors can intervene before a medical emergency occurs.
🔹 Parameters Collected (Every Hour)
Feature	Description
heart_rate	Beats per minute
systolic_bp	Blood pressure (mmHg)
oxygen	Oxygen saturation (%)
resp_rate	Breaths per minute
temperature	Body temperature (°C)

In [47]:

import numpy as np
import pandas as pd

np.random.seed(42)

In [5]:
from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras import Sequential
import pandas as pd
import numpy as np

df = pd.read_csv("icu_timseries_12Hrs.csv")

In [81]:
model = Sequential()
model.add(Input(shape=(12, 5)))
# model.add(SimpleRNN(8, activation="tanh", return_sequences=True))
model.add(SimpleRNN(16, activation="tanh"))
model.add(Dense(6, activation="linear"))

model.compile(optimizer="SGD", loss="mse", metrics=["mae"])
model.summary()


Model: "sequential_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_34 (SimpleRNN)       │ (None, 16)             │           352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 6)              │           102 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 454 (1.77 KB)

 Trainable params: 454 (1.77 KB)

 Non-trainable params: 0 (0.00 B)

In [82]:


rows = 72   # total hours (enough for training)

data = {
    "HR":    np.random.normal(90, 10, rows),    # heart rate
    "BP":    np.random.normal(110, 15, rows),   # blood pressure
    "O2":    np.random.normal(95, 2, rows),     # oxygen saturation
    "Resp":  np.random.normal(18, 3, rows),     # respiratory rate
    "Temp":  np.random.normal(37, 0.5, rows)    # body temperature
}

df = pd.DataFrame(data)

# Save to CSV
df.to_csv("icu_timseries_12Hrs.csv", index=False)

print("Synthetic ICU data saved:", df.shape)
print(df.head())


Synthetic ICU data saved: (72, 5)
           HR          BP         O2       Resp       Temp
0   82.683683   92.732694  94.874721  17.972134  37.246493
1  108.904407   67.413820  95.486247  18.920943  36.971241
2   90.493083   97.859288  94.411515  23.440223  37.537672
3   97.688397  124.350548  97.713167  16.971363  37.105334
4   83.917735  117.754198  94.599295  18.835046  37.104767


In [83]:

data = df[["HR", "BP", "O2", "Resp", "Temp"]].values.astype("float32")

X, y = [], []
for i in range(12, len(data) - 5):
    X.append(data[i-12:i])
    y.append(data[i:i+6, 4])   

X = np.array(X)       
y = np.array(y)      

model.fit(X, y, epochs=80, batch_size=12, validation_split=0.2, shuffle=False)


Epoch 1/80
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 1168.5171 - mae: 34.0904 - val_loss: 857.1351 - val_mae: 29.2537
Epoch 2/80
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 733.6658 - mae: 27.0099 - val_loss: 538.3549 - val_mae: 23.1825
Epoch 3/80
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 460.3215 - mae: 21.3921 - val_loss: 338.2892 - val_mae: 18.3749
Epoch 4/80
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 288.8781 - mae: 16.9435 - val_loss: 212.7011 - val_mae: 14.5679
Epoch 5/80
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 181.3419 - mae: 13.4208 - val_loss: 133.8437 - val_mae: 11.5531
Epoch 6/80
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 113.8863 - mae: 10.6312 - val_loss: 84.3116 - val_mae: 9.1658
Epoch 7/80
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 71.5690 - mae: 8.4221 - val_loss: 53.1860 - val_mae: 7.2753
Epoch 8/80
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 45.0188 - mae: 6.6728 - val_loss: 33.6162 - val_mae: 5.7783
Epoch 9/80
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18